## S&P500 index constituents and their identifiers from CRSP 
- This programming file was originally downloaded from WRDS and modified.

- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [1]:
pip install wrds

Note: you may need to restart the kernel to use updated packages.


In [2]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Step 1: Connect to WRDS

In [3]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Enter your WRDS username [anthony_ning]: dr1902
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  n


You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [4]:
# Define the SQL query
query = """
    SELECT a.permno, a.start, a.ending, b.date, b.ret
    FROM crsp.msp500list AS a
    JOIN crsp.msf AS b
    ON a.permno = b.permno
    WHERE b.date >= a.start
      AND b.date <= a.ending
      AND b.date >= '2014-01-01'
    ORDER BY b.date;
"""

# Execute the SQL query
sp500 = conn.raw_sql(query, date_cols=['start', 'ending', 'date'])

# Display the last few rows of the result
sp500.tail()

,permno,start,ending,date,ret
60483,61241,2017-03-20,2023-12-29,2023-12-29,0.216656
60484,24174,2023-10-02,2023-12-29,2023-12-29,0.066019
60485,15069,1925-12-31,2023-12-29,2023-12-29,-0.049941
60486,86339,2012-03-14,2023-12-29,2023-12-29,-0.004476
60487,45751,1987-08-06,2023-12-29,2023-12-29,-0.049895


In [5]:
# Fetch only quarter-end data
is_quatend = []

for row in sp500.iterrows():
    if row[1][3].month == 3 or row[1][3].month == 6 or row[1][3].month == 9 or row[1][3].month == 12:
        is_quatend.append(True)
    else:
        is_quatend.append(False)

sp500 = sp500[is_quatend]
sp500.head()

/var/folders/60/75vh_thj2m57nynzg5tgw0300000gn/T/ipykernel_54681/2957911958.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[1][3].month == 3 or row[1][3].month == 6 or row[1][3].month == 9 or row[1][3].month == 12:


,permno,start,ending,date,ret
1000,90441,2004-12-20,2019-03-19,2014-03-31,-0.043083
1001,21776,1944-06-07,2023-12-29,2014-03-31,0.103584
1002,22111,1973-05-31,2023-12-29,2014-03-31,0.066327
1003,80100,2002-06-26,2023-12-29,2014-03-31,0.016802
1004,22103,1965-01-07,2023-12-29,2014-03-31,0.023598


In [6]:
sp500.tail()

,permno,start,ending,date,ret
60483,61241,2017-03-20,2023-12-29,2023-12-29,0.216656
60484,24174,2023-10-02,2023-12-29,2023-12-29,0.066019
60485,15069,1925-12-31,2023-12-29,2023-12-29,-0.049941
60486,86339,2012-03-14,2023-12-29,2023-12-29,-0.004476
60487,45751,1987-08-06,2023-12-29,2023-12-29,-0.049895


### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [7]:
# Add Other Descriptive Variables
query = """
    SELECT comnam, namedt, nameendt, permno, ticker
    FROM crsp.msenames
"""

mse = conn.raw_sql(query, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))
mse.head(5)

,comnam,namedt,nameendt,permno,ticker
0,OPTIMUM MANUFACTURING INC,1986-01-07,1986-12-03,10000,OMFGA
1,OPTIMUM MANUFACTURING INC,1986-12-04,1987-03-09,10000,OMFGA
2,OPTIMUM MANUFACTURING INC,1987-03-10,1987-06-11,10000,OMFGA
3,GREAT FALLS GAS CO,1986-01-09,1993-11-21,10001,GFGC
4,ENERGY WEST INC,1993-11-22,2004-06-09,10001,EWST


In [10]:
mse.tail(5)

,comnam,namedt,nameendt,permno,ticker
113851,S & W SEED CO,2010-06-14,2023-12-29,93434,SANW
113852,SINO CLEAN ENERGY INC,2010-06-14,2012-05-18,93435,SCEI
113853,TESLA MOTORS INC,2010-06-29,2017-02-01,93436,TSLA
113854,TESLA INC,2017-02-02,2023-04-02,93436,TSLA
113855,TESLA INC,2023-04-03,2023-12-29,93436,TSLA


In [8]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]

In [27]:
sp500_full.head(5)

,permno,start,ending,date,ret,comnam,namedt,nameendt,ticker
3,90441,2004-12-20,2019-03-19,2014-03-31,-0.043083,TWENTY FIRST CENTURY FOX INC,2013-08-02,2019-03-18,FOXA
16,21776,1944-06-07,2023-12-29,2014-03-31,0.103584,EXELON CORP,2014-02-06,2016-12-18,EXC
26,22111,1973-05-31,2023-12-29,2014-03-31,0.066327,JOHNSON & JOHNSON,2006-07-11,2021-12-20,JNJ
34,80100,2002-06-26,2023-12-29,2014-03-31,0.016802,SIMON PROPERTY GROUP INC NEW,2010-12-13,2020-02-10,SPG
43,22103,1965-01-07,2023-12-29,2014-03-31,0.023598,EMERSON ELECTRIC CO,2014-01-08,2017-01-10,EMR


In [28]:
sp500_full.tail(5)

,permno,start,ending,date,ret,comnam,namedt,nameendt,ticker
120688,61241,2017-03-20,2023-12-29,2023-12-29,0.216656,ADVANCED MICRO DEVICES INC,2015-01-02,2023-12-29,AMD
120689,24174,2023-10-02,2023-12-29,2023-12-29,0.066019,VERALTO CORP,2023-10-02,2023-12-29,VLTO
120704,15069,1925-12-31,2023-12-29,2023-12-29,-0.049941,MARATHON OIL CORP,2021-04-01,2023-12-29,MRO
120717,86339,2012-03-14,2023-12-29,2023-12-29,-0.004476,CROWN CASTLE INC,2022-08-02,2023-12-29,CCI
120725,45751,1987-08-06,2023-12-29,2023-12-29,-0.049895,MARSH & MCLENNAN COS INC,2019-09-12,2023-12-29,MMC


In [9]:
cnt = sp500_full.groupby(['date']).count()
cnt.head(4)

,permno,start,ending,ret,comnam,namedt,nameendt,ticker
date,,,,,,,,
2014-03-31,500,500,500,500,500,500,500,500
2014-06-30,501,501,501,501,501,501,501,501
2014-09-30,502,502,502,502,502,502,502,502
2014-12-31,502,502,502,502,502,502,502,502


In [30]:
cnt.tail()

,permno,start,ending,ret,comnam,namedt,nameendt,ticker
date,,,,,,,,
2022-09-30,502,502,502,502,502,502,502,502
2022-12-30,503,503,503,503,503,503,503,503
2023-03-31,503,503,503,503,503,503,503,503
2023-06-30,503,503,503,503,503,503,503,503
2023-12-29,503,503,503,503,503,503,503,503


In [10]:
# Write data into csv format
sp500_full = sp500_full.drop(columns=['start', 'ending', 'ret', 'namedt', 'nameendt'])
sp500_full.to_csv('sp500_composition.csv', index=False)

### Earnings Estimates of S&P500
Based on the composition of S&P500 from 2023Q4, we list the history of earnings estimates for the next fiscal quarter announced during 2024. 

In [32]:
d = pd.Timestamp(2023, 12, 29)
sp500_today = sp500_full.loc[sp500_full.date == d]
sp500_today = sp500_today.drop(columns=['date'])
sp500_today.head()

,comnam,ticker
117204,INTEL CORP,INTC
117210,WILLIS TOWERS WATSON PUB LTD CO,WTW
117215,LOWES COMPANIES INC,LOW
117218,QUEST DIAGNOSTICS INC,DGX
117220,KEURIG DR PEPPER INC,KDP


In [22]:
query = """
    SELECT ticker, anndats_act, fpi, meanest, medest, highest, lowest
    FROM ibes.statsum_epsus
    WHERE fpi = '6'
    AND anndats_act BETWEEN '2024-01-01' AND '2024-12-30'
"""
# Query the ibes_detail data for a specific company and time period
data = conn.raw_sql(query)

# Display the data
data.head()

,ticker,anndats_act,fpi,meanest,medest,highest,lowest
0,000Y,2024-02-15,6,-1.62,-1.65,-0.71,-2.45
1,000Y,2024-02-15,6,-1.60,-1.59,-0.71,-2.45
2,000Y,2024-02-15,6,-1.61,-1.59,-0.71,-2.45
3,002S,2024-02-07,6,-0.08,-0.09,-0.02,-0.12
4,002S,2024-02-07,6,-0.08,-0.09,-0.02,-0.12


In [33]:
sp_earnings_est = pd.merge(sp500_today, data, how = 'left', on = 'ticker')
sp_earnings_est.head()

,comnam,ticker,anndats_act,fpi,meanest,medest,highest,lowest
0,INTEL CORP,INTC,2024-01-25,6,0.45,0.44,0.54,0.43
1,INTEL CORP,INTC,2024-01-25,6,0.45,0.44,0.54,0.43
2,INTEL CORP,INTC,2024-01-25,6,0.45,0.44,0.54,0.44
3,WILLIS TOWERS WATSON PUB LTD CO,WTW,NaN,NaN,NaN,NaN,NaN,NaN
4,LOWES COMPANIES INC,LOW,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
sp_earnings_est.to_csv('sp500_earnings_est.csv', index=False)